In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from densenet_train import *

In [2]:
# Config
config = {
    'growth_rate': 12,
    'num_blocks': 3,
    'layers_per_block': 4,
    'init_channels': 24,
    'compression': 0.5,
    'dropout_rate': 0.1,
    'bottleneck': True,
    'num_classes': 10
}

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# CIFAR-10 Data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, transform=transform)

# Split train/val
train_set, val_set = random_split(train_data, [45000, 5000])
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

100%|██████████| 170498071/170498071 [00:07<00:00, 24079285.02it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data


In [4]:
# Model
model = DenseNet(config).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train
model.fit(train_loader, val_loader, optimizer, criterion, device)

# Save model + plot
model.save_model("best_densenet_cifar10.pth")
model.visualize_training("cifar10_training_plot.png")

# Final test
test_acc = model.evaluate(test_loader, device)
print(f"🎯 Final Test Accuracy: {test_acc:.4f}")

Epoch 01 | Loss: 1.5782 | Train Acc: 0.4148 | Val Acc: 0.4930


KeyboardInterrupt: 